<a href="https://colab.research.google.com/github/sylvia31096/Secure-and-Private-AI/blob/master/Differential_Privacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Creating my own private database

In [0]:
import torch

In [0]:
num_records = 5000 #number of 5000 records

Create a function to create the database and parallel databases

In [0]:
 def get_parallelDB(db,removable_index):

    return torch.cat((db[0:removable_index],db[removable_index+1:]))

Create function to make a parallel database

In [0]:
def get_parallelDBs(db):

    parallelDBs = []
    for i in range(len(db)):
      parallelDB = get_parallelDB(db,i)
      parallelDBs.append(parallelDB)


    return parallelDBs

In [0]:
def create_db_and_parallels(num_records):
  db = torch.rand(num_records)>0.5
  parallelDBs =  get_parallelDBs(db)
  
  return db,parallelDBs

## Evaluating differential privacy

Getting sensitivty

In [0]:
db, pdbs = create_db_and_parallels(5000)

In [0]:
def query_mean(db):
    return db.float().mean()

In [0]:
full_db_result = query(db)

In [0]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance

In [10]:
sensitivity

tensor(0.0001)

Sensitivity function

In [0]:
def sensitivity(query,n_entries):
    db, pdbs = create_db_and_parallels(n_entries)
    
    full_db_result = query(db)
    
    
    max_distance = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > max_distance):
            max_distance = db_distance
    return max_distance

In [0]:
def query_threshold(db,threshold=5):
  return (db.sum()>threshold).float()


def query_sum(db):
  return db.float().sum()

In [13]:
sensitivity(query,5000)

0

In [14]:
for i in range (10):
  print(sensitivity(query,10))

tensor(1.)
0
0
tensor(1.)
0
0
0
0
tensor(1.)
0


### Performing a diffferentiation attack

To do so:
1. query the database 
2. query database without John Doe

In [0]:
db,without_john_doe = create_db_and_parallels(100)

Let's query using sum

In [21]:
query_sum(db)

tensor(53.)

In [22]:
query_sum(without_john_doe[10])

tensor(53.)

Its 0

Let's query using mean

In [25]:
query_mean(db)

tensor(0.5300)

In [26]:
query_mean(without_john_doe[10])

tensor(0.5354)

Its 0

Let's query using L1

In [27]:
query_threshold(db)

tensor(1.)

In [28]:
query_threshold(without_john_doe[10])

tensor(1.)

Not sure